In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train.shape, test.shape

((250000, 102), (150000, 101))

In [3]:
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [4]:
test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
1,250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
2,250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
3,250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
4,250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [6]:
kf = KFold(n_splits = 15, shuffle = False)

In [7]:
skf = StratifiedKFold(n_splits = 15, shuffle = False)

In [8]:
X = train.iloc[:, 1:-1]
y = train['loss']

target = test.iloc[:, 1:]

In [9]:
scaler = StandardScaler()

In [10]:
scaled_x = scaler.fit_transform(X)

In [11]:
target = scaler.transform(target)

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
xgb_pred = np.zeros((target.shape[0]))
xgb_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    xgb = XGBRegressor(n_estimators = 10000, learning_rate = .005, objective = 'reg:squarederror', random_state = 42, n_jobs = -1)
    
    
    xgb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 2500, early_stopping_rounds = 1000)
    val_pred = xgb.predict(val_x)
    rmse = np.sqrt(mean_squared_error(val_y, val_pred))
    xgb_rmse.append(rmse)
    
    pred = xgb.predict(target) / 15
    
    xgb_pred += pred
print(f'\nRMSE of XGBoost is {np.mean(xgb_rmse)}')

In [24]:
cb_pred = np.zeros((target.shape[0]))
cb_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    cb = CatBoostRegressor(loss_function = 'RMSE', eval_metric = 'RMSE', silent = True, iterations = 10000, learning_rate = .005, random_state = 42)
    
    train_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 1000, use_best_model = True, verbose = 2500)
    rmse = cb.best_score_['validation']['RMSE']
    cb_rmse.append(rmse)
    pred = cb.predict(target) / 15
    
    cb_pred += pred
print(f'\nRMSE of CatBoost is {np.mean(cb_rmse)}')

0:	learn: 7.9443303	test: 7.8790053	best: 7.8790053 (0)	total: 97.3ms	remaining: 16m 12s
2500:	learn: 7.8373374	test: 7.8213429	best: 7.8213429 (2500)	total: 53.2s	remaining: 2m 39s
5000:	learn: 7.7653065	test: 7.8064105	best: 7.8064105 (5000)	total: 1m 44s	remaining: 1m 44s
7500:	learn: 7.7015323	test: 7.7990985	best: 7.7990934 (7499)	total: 2m 34s	remaining: 51.4s
9999:	learn: 7.6414627	test: 7.7943013	best: 7.7942876 (9939)	total: 3m 26s	remaining: 0us

bestTest = 7.794287574
bestIteration = 9939

Shrink model to first 9940 iterations.
0:	learn: 7.9429378	test: 7.8985799	best: 7.8985799 (0)	total: 33.3ms	remaining: 5m 33s
2500:	learn: 7.8375339	test: 7.8249053	best: 7.8249053 (2500)	total: 55.7s	remaining: 2m 46s
5000:	learn: 7.7653144	test: 7.8064777	best: 7.8064776 (4999)	total: 1m 56s	remaining: 1m 56s
7500:	learn: 7.7010651	test: 7.7985743	best: 7.7985743 (7500)	total: 2m 54s	remaining: 58.1s
9999:	learn: 7.6407820	test: 7.7935926	best: 7.7935812 (9997)	total: 3m 48s	remaining: 

In [25]:
lgbm_pred = np.zeros((target.shape[0]))
lgbm_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    lgbm = LGBMRegressor(n_estimators = 10000, learning_rate = .005, objective = 'regression', metric = 'rmse', random_state = 42, n_jobs = -1)
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 2500, early_stopping_rounds = 1000)
    rmse = np.sqrt(list(lgbm.best_score_['valid_1'].values())[0])
    lgbm_rmse.append(rmse)
    
    pred = lgbm.predict(target) / 15
    
    lgbm_pred += pred
print(f'\nRMSE of LGBM is {np.mean(lgbm_rmse)}')

Training until validation scores don't improve for 1000 rounds
[2500]	training's rmse: 7.67263	valid_1's rmse: 7.80434
[5000]	training's rmse: 7.4975	valid_1's rmse: 7.79978
Early stopping, best iteration is:
[4828]	training's rmse: 7.50894	valid_1's rmse: 7.79955
Training until validation scores don't improve for 1000 rounds
[2500]	training's rmse: 7.6728	valid_1's rmse: 7.80428
[5000]	training's rmse: 7.4983	valid_1's rmse: 7.79653
Early stopping, best iteration is:
[6034]	training's rmse: 7.43091	valid_1's rmse: 7.79599
Training until validation scores don't improve for 1000 rounds
[2500]	training's rmse: 7.66863	valid_1's rmse: 7.87166
[5000]	training's rmse: 7.49596	valid_1's rmse: 7.86574
Early stopping, best iteration is:
[6287]	training's rmse: 7.41349	valid_1's rmse: 7.86449
Training until validation scores don't improve for 1000 rounds
[2500]	training's rmse: 7.66589	valid_1's rmse: 7.90252
[5000]	training's rmse: 7.49275	valid_1's rmse: 7.8979
[7500]	training's rmse: 7.33277

In [29]:
from sklearn.neural_network import MLPRegressor

In [34]:
mlp = MLPRegressor(activation = 'relu', random_state = 42, batch_size = 256)

In [36]:
mlp_pred = np.zeros((target.shape[0]))
mlp_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    mlp = MLPRegressor(activation = 'relu', random_state = 42, batch_size = 256)
    
    mlp.fit(tr_x, tr_y)
    val_pred = mlp.predict(val_x)
    rmse = np.sqrt(mean_squared_error(val_y, val_pred))
    mlp_rmse.append(rmse)
    
    pred = mlp.predict(target) / 15
    
    mlp_pred += pred
print(f'\nRMSE of mlp is {np.mean(mlp_rmse)}')

KeyboardInterrupt: 

***

## Make a submission 

In [27]:
submission['loss'] = lgbm_pred * 0.3 + cb_pred * .7

In [28]:
submission.to_csv('2nd.csv', index = False)